# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [306]:
problem = '''We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number. 

We want to maximize the daily profit of this alkylation process. 
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup. 

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas. 
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed. 
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate. 
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup.'''

## 2. Ask for parameters

In [307]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [308]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

**Decision Variables:**

*  **F_o:** Olefin feed (barrels per day)
*  **F_iR:** Isobutane recycle (barrels per day)
*  **F_aA:** Acid addition rate (thousands of pounds per day) 
*  **F_iM:** Isobutane makeup (barrels per day)

**Dependent Variables:**

*  **Y_Alk:** Alkylate yield (barrels per day)
*  **MON:** Motor octane number
*  **DF:** Acid dilution factor
*  **F4:** F-4 performance number
*  **R_i/o:** External isobutane-to-olefin ratio
*  **C_A:** Acid strength (weight percent) 


# 2. Ask for objective

In [309]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [310]:
Markdown(response2.text)

## Objective Function:

Maximize Profit (P)

**P = (Y_Alk * MON * ε_Alk) - (C_o * F_o + C_i * (F_iR + F_iM) + C_aA * F_aA)** 

**Where:**

* **P:** Daily profit
* **Y_Alk:** Alkylate yield (calculated from regression formula, subject to 2% deviation: Y_Alk_actual = Y_Alk ± 0.02 * Y_Alk)
* **MON:** Motor octane number (calculated from regression formula, subject to 1% deviation: MON_actual = MON ± 0.01 * MON)
* **ε_Alk:** Revenue per barrel of alkylate (dependent on MON, pricing not specified)
* **C_o:** Cost per barrel of olefin feed
* **C_i:** Cost per barrel of isobutane 
* **C_aA:** Cost per thousand pounds of acid addition 
* **F_o:** Olefin feed (barrels per day)
* **F_iR:** Isobutane recycle (barrels per day)
* **F_iM:** Isobutane makeup (barrels per day)
* **F_aA:** Acid addition rate (thousands of pounds per day) 


# 3. Ask for constraints

In [311]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [312]:
Markdown(response3.text)

## Constraints:

**Regression Model Constraints (with allowable deviations):**

1.  **Alkylate Yield:**
    *   Lower Bound:  Y_Alk * (1 - 0.02)  ≤  Y_Alk_actual = F_o * (1.12 + 0.13167 * R_i/o - 0.00667 * R_i/o^2) 
    *   Upper Bound:  Y_Alk * (1 + 0.02)  ≥  Y_Alk_actual = F_o * (1.12 + 0.13167 * R_i/o - 0.00667 * R_i/o^2) 

2.  **Motor Octane Number:**
    *   Lower Bound:  MON * (1 - 0.01) ≤ MON_actual = 86.35 + 1.098 * R_i/o - 0.038 * R_i/o^2 + 0.325 * (C_A - 89)
    *   Upper Bound:  MON * (1 + 0.01) ≥ MON_actual = 86.35 + 1.098 * R_i/o - 0.038 * R_i/o^2 + 0.325 * (C_A - 89)

3.  **Acid Dilution Factor:**
    *   Lower Bound:  DF * (1 - 0.05) ≤ DF_actual = 35.82 - 0.222 * F4 
    *   Upper Bound:  DF * (1 + 0.05) ≥ DF_actual = 35.82 - 0.222 * F4

4.  **F-4 Performance Number:**
    *   Lower Bound:  F4 * (1 - 0.10) ≤ F4_actual = -133 + 3 * MON
    *   Upper Bound:  F4 * (1 + 0.10) ≥ F4_actual = -133 + 3 * MON

**Process Constraints:**

5.  **External Isobutane-to-Olefin Ratio:** 
    *   R_i/o = (F_iR + F_iM) / F_o

6.  **Acid Strength:**
    *   C_A = (93000 * F_aA) / (F_aA * DF + 1000 * F_aA)

7.  **Alkylate Yield (Process Relationship):**
    *   1.22 * Y_Alk =  F_o + F_iM 

**Variable Bounds:**

8.  **Lower and Upper Bounds:** 
    *   F_o_min ≤ F_o ≤ F_o_max
    *   F_iR_min ≤ F_iR ≤ F_iR_max
    *   F_aA_min ≤ F_aA ≤ F_aA_max
    *   F_iM_min ≤ F_iM ≤ F_iM_max

**Where:** _min and _max represent the minimum and maximum allowable values for each variable, respectively. These values are not specified in the problem statement._ 


## 3. Generate the pyomo code

In [313]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [314]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with actual data if available)
data = {
    'ε_Alk': 2.5,  # Sample revenue per barrel of alkylate (replace with actual pricing)
    'C_o': 50,     # Sample cost per barrel of olefin feed
    'C_i': 45,     # Sample cost per barrel of isobutane
    'C_aA': 0.2,   # Sample cost per thousand pounds of acid addition
    'F_o_min': 10000,  # Sample minimum olefin feed
    'F_o_max': 20000,  # Sample maximum olefin feed
    'F_iR_min': 5000,   # Sample minimum isobutane recycle
    'F_iR_max': 15000,  # Sample maximum isobutane recycle
    'F_aA_min': 50,     # Sample minimum acid addition rate
    'F_aA_max': 150,    # Sample maximum acid addition rate
    'F_iM_min': 1000,   # Sample minimum isobutane makeup
    'F_iM_max': 10000,  # Sample maximum isobutane makeup
}

# Model Creation
model = pyo.ConcreteModel()

# Decision Variables
model.F_o = pyo.Var(bounds=(data['F_o_min'], data['F_o_max']))
model.F_iR = pyo.Var(bounds=(data['F_iR_min'], data['F_iR_max']))
model.F_aA = pyo.Var(bounds=(data['F_aA_min'], data['F_aA_max']))
model.F_iM = pyo.Var(bounds=(data['F_iM_min'], data['F_iM_max']))

# Dependent Variables
model.Y_Alk = pyo.Var()
model.MON = pyo.Var()
model.DF = pyo.Var()
model.F4 = pyo.Var()
model.R_i_o = pyo.Var()
model.C_A = pyo.Var()

# Objective Function
def profit(model):
    return (model.Y_Alk * model.MON * data['ε_Alk']) - (data['C_o'] * model.F_o + data['C_i'] * (model.F_iR + model.F_iM) + data['C_aA'] * model.F_aA)
model.P = pyo.Objective(rule=profit, sense=pyo.maximize)

# Constraints
model.Alkylate_Yield_Lower = pyo.Constraint(expr=model.Y_Alk * (1 - 0.02) <= model.F_o * (1.12 + 0.13167 * model.R_i_o - 0.00667 * model.R_i_o**2))
model.Alkylate_Yield_Upper = pyo.Constraint(expr=model.Y_Alk * (1 + 0.02) >= model.F_o * (1.12 + 0.13167 * model.R_i_o - 0.00667 * model.R_i_o**2))

model.MON_Lower = pyo.Constraint(expr=model.MON * (1 - 0.01) <= 86.35 + 1.098 * model.R_i_o - 0.038 * model.R_i_o**2 + 0.325 * (model.C_A - 89))
model.MON_Upper = pyo.Constraint(expr=model.MON * (1 + 0.01) >= 86.35 + 1.098 * model.R_i_o - 0.038 * model.R_i_o**2 + 0.325 * (model.C_A - 89))

model.DF_Lower = pyo.Constraint(expr=model.DF * (1 - 0.05) <= 35.82 - 0.222 * model.F4)
model.DF_Upper = pyo.Constraint(expr=model.DF * (1 + 0.05) >= 35.82 - 0.222 * model.F4)

model.F4_Lower = pyo.Constraint(expr=model.F4 * (1 - 0.10) <= -133 + 3 * model.MON)
model.F4_Upper = pyo.Constraint(expr=model.F4 * (1 + 0.10) >= -133 + 3 * model.MON)

model.External_Isobutane_Ratio = pyo.Constraint(expr=model.R_i_o == (model.F_iR + model.F_iM) / model.F_o)
model.Acid_Strength = pyo.Constraint(expr=model.C_A == (93000 * model.F_aA) / (model.F_aA * model.DF + 1000 * model.F_aA))
model.Alkylate_Yield_Process = pyo.Constraint(expr=1.22 * model.Y_Alk == model.F_o + model.F_iM)

# Solve the Model
solver = pyo.SolverFactory('ipopt') # Replace 'ipopt' with another solver if desired
results = solver.solve(model)

# Print the Results
print("Status:", results.solver.status)
print("Termination condition:", results.solver.termination_condition)
print("Optimal Solution:")
print("F_o (Olefin Feed):", pyo.value(model.F_o))
print("F_iR (Isobutane Recycle):", pyo.value(model.F_iR))
print("F_aA (Acid Addition Rate):", pyo.value(model.F_aA))
print("F_iM (Isobutane Makeup):", pyo.value(model.F_iM))
print("Profit:", pyo.value(model.P)) 
```

This code defines the optimization problem in Pyomo, including the objective function, decision variables, constraints, and sample data. Remember to replace the sample data with your actual data for meaningful results. 

The code uses the 'ipopt' solver, but you can choose a different solver if needed. After solving, it prints the solver status, termination condition, optimal values of the decision variables, and the maximized profit. 


## 4. Input problem data and try running the generated code

In [316]:
import pyomo.environ as pyo

# Costs
c1, c2, c3, c4, c5 = 0.63, 5.04, 0.035, 10.0, 3.36

# Lower and upper bounds
x1_l, x1_u = (0, 2000)
x2_l, x2_u = (0, 16000)
x3_l, x3_u = (0, 120)
x4_l, x4_u = (0, 5000)
x5_l, x5_u = (0, 2000)
x6_l, x6_u = (85, 93)
x7_l, x7_u = (90, 95)
x8_l, x8_u = (3, 12)
x9_l, x9_u = (1.2, 4)
x10_l, x10_u = (145, 162)

# Decision Variables:
#     x1 (continuous): Olefin feed (barrels per day)
#     x2 (continuous): Isobutane recycle (barrels per day)
#     x3 (continuous): Acid addition rate (thousands of pounds per day)
#     x4 (continuous): Alkylate yield (barrels per day)
#     x5 (continuous): Isobutane makeup (barrels per day)
#     x6 (continuous): Acid strength (weight percent)
#     x7 (continuous): Motor octane number
#     x8 (continuous): External isobutane-to-olefin ratio
#     x9 (continuous): Acid dilution factor
#     x10 (continuous): F-4 performance number

# Parameters:
#     c1 to c5: Costs (used in the objective function)
#     x_i^l: Lower bound of variable i
#     x_i^u: Upper bound of variable i
# Sample Data (replace with actual data if available)
data = {
    'ε_Alk': 0.63,  # Sample revenue per barrel of alkylate (replace with actual pricing)
    'C_o': 5.04,     # Sample cost per barrel of olefin feed
    'C_i': 0.035,     # Sample cost per barrel of isobutane
    'C_aA': 10,   # Sample cost per thousand pounds of acid addition
    'F_o_min': 0,  # Sample minimum olefin feed
    'F_o_max': 2000,  # Sample maximum olefin feed
    'F_iR_min': 0,   # Sample minimum isobutane recycle
    'F_iR_max': 16000,  # Sample maximum isobutane recycle
    'F_aA_min': 0,     # Sample minimum acid addition rate
    'F_aA_max': 120,    # Sample maximum acid addition rate
    'F_iM_min': 0,   # Sample minimum isobutane makeup
    'F_iM_max': 5000,  # Sample maximum isobutane makeup
}

# Model Creation
model = pyo.ConcreteModel()

# Decision Variables
model.F_o = pyo.Var(bounds=(data['F_o_min'], data['F_o_max']))
model.F_iR = pyo.Var(bounds=(data['F_iR_min'], data['F_iR_max']))
model.F_aA = pyo.Var(bounds=(data['F_aA_min'], data['F_aA_max']))
model.F_iM = pyo.Var(bounds=(data['F_iM_min'], data['F_iM_max']))

# Dependent Variables
model.Y_Alk = pyo.Var()
model.MON = pyo.Var()
model.DF = pyo.Var()
model.F4 = pyo.Var()
model.R_i_o = pyo.Var()
model.C_A = pyo.Var()

# Objective Function
def profit(model):
    return (model.Y_Alk * model.MON * data['ε_Alk']) - (data['C_o'] * model.F_o + data['C_i'] * (model.F_iR + model.F_iM) + data['C_aA'] * model.F_aA)
model.P = pyo.Objective(rule=profit, sense=pyo.maximize)

# Constraints
model.Alkylate_Yield_Lower = pyo.Constraint(expr=model.Y_Alk * (1 - 0.02) <= model.F_o * (1.12 + 0.13167 * model.R_i_o - 0.00667 * model.R_i_o**2))
model.Alkylate_Yield_Upper = pyo.Constraint(expr=model.Y_Alk * (1 + 0.02) >= model.F_o * (1.12 + 0.13167 * model.R_i_o - 0.00667 * model.R_i_o**2))

model.MON_Lower = pyo.Constraint(expr=model.MON * (1 - 0.01) <= 86.35 + 1.098 * model.R_i_o - 0.038 * model.R_i_o**2 + 0.325 * (model.C_A - 89))
model.MON_Upper = pyo.Constraint(expr=model.MON * (1 + 0.01) >= 86.35 + 1.098 * model.R_i_o - 0.038 * model.R_i_o**2 + 0.325 * (model.C_A - 89))

model.DF_Lower = pyo.Constraint(expr=model.DF * (1 - 0.05) <= 35.82 - 0.222 * model.F4)
model.DF_Upper = pyo.Constraint(expr=model.DF * (1 + 0.05) >= 35.82 - 0.222 * model.F4)

model.F4_Lower = pyo.Constraint(expr=model.F4 * (1 - 0.10) <= -133 + 3 * model.MON)
model.F4_Upper = pyo.Constraint(expr=model.F4 * (1 + 0.10) >= -133 + 3 * model.MON)

model.External_Isobutane_Ratio = pyo.Constraint(expr=model.R_i_o == (model.F_iR + model.F_iM) / model.F_o)
model.Acid_Strength = pyo.Constraint(expr=model.C_A == (93000 * model.F_aA) / (model.F_aA * model.DF + 1000 * model.F_aA))
model.Alkylate_Yield_Process = pyo.Constraint(expr=1.22 * model.Y_Alk == model.F_o + model.F_iM)

# Solve the Model
solver = pyo.SolverFactory('ipopt') # Replace 'ipopt' with another solver if desired
results = solver.solve(model)

# Print the Results
print("Status:", results.solver.status)
print("Termination condition:", results.solver.termination_condition)
print("Optimal Solution:")
print("F_o (Olefin Feed):", pyo.value(model.F_o))
print("F_iR (Isobutane Recycle):", pyo.value(model.F_iR))
print("F_aA (Acid Addition Rate):", pyo.value(model.F_aA))
print("F_iM (Isobutane Makeup):", pyo.value(model.F_iM))
print("Profit:", pyo.value(model.P)) 

Status: ok
Termination condition: optimal
Optimal Solution:
F_o (Olefin Feed): 2000.0
F_iR (Isobutane Recycle): 16000.0
F_aA (Acid Addition Rate): 0.0
F_iM (Isobutane Makeup): 2398.9981404595396
Profit: 206192.39364181433


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [317]:
print(response.text)

## Variables:

**Decision Variables:**

*  **F_o:** Olefin feed (barrels per day)
*  **F_iR:** Isobutane recycle (barrels per day)
*  **F_aA:** Acid addition rate (thousands of pounds per day) 
*  **F_iM:** Isobutane makeup (barrels per day)

**Dependent Variables:**

*  **Y_Alk:** Alkylate yield (barrels per day)
*  **MON:** Motor octane number
*  **DF:** Acid dilution factor
*  **F4:** F-4 performance number
*  **R_i/o:** External isobutane-to-olefin ratio
*  **C_A:** Acid strength (weight percent) 



In [318]:
print(response2.text)

## Objective Function:

Maximize Profit (P)

**P = (Y_Alk * MON * ε_Alk) - (C_o * F_o + C_i * (F_iR + F_iM) + C_aA * F_aA)** 

**Where:**

* **P:** Daily profit
* **Y_Alk:** Alkylate yield (calculated from regression formula, subject to 2% deviation: Y_Alk_actual = Y_Alk ± 0.02 * Y_Alk)
* **MON:** Motor octane number (calculated from regression formula, subject to 1% deviation: MON_actual = MON ± 0.01 * MON)
* **ε_Alk:** Revenue per barrel of alkylate (dependent on MON, pricing not specified)
* **C_o:** Cost per barrel of olefin feed
* **C_i:** Cost per barrel of isobutane 
* **C_aA:** Cost per thousand pounds of acid addition 
* **F_o:** Olefin feed (barrels per day)
* **F_iR:** Isobutane recycle (barrels per day)
* **F_iM:** Isobutane makeup (barrels per day)
* **F_aA:** Acid addition rate (thousands of pounds per day) 



In [319]:
print(response3.text)

## Constraints:

**Regression Model Constraints (with allowable deviations):**

1.  **Alkylate Yield:**
    *   Lower Bound:  Y_Alk * (1 - 0.02)  ≤  Y_Alk_actual = F_o * (1.12 + 0.13167 * R_i/o - 0.00667 * R_i/o^2) 
    *   Upper Bound:  Y_Alk * (1 + 0.02)  ≥  Y_Alk_actual = F_o * (1.12 + 0.13167 * R_i/o - 0.00667 * R_i/o^2) 

2.  **Motor Octane Number:**
    *   Lower Bound:  MON * (1 - 0.01) ≤ MON_actual = 86.35 + 1.098 * R_i/o - 0.038 * R_i/o^2 + 0.325 * (C_A - 89)
    *   Upper Bound:  MON * (1 + 0.01) ≥ MON_actual = 86.35 + 1.098 * R_i/o - 0.038 * R_i/o^2 + 0.325 * (C_A - 89)

3.  **Acid Dilution Factor:**
    *   Lower Bound:  DF * (1 - 0.05) ≤ DF_actual = 35.82 - 0.222 * F4 
    *   Upper Bound:  DF * (1 + 0.05) ≥ DF_actual = 35.82 - 0.222 * F4

4.  **F-4 Performance Number:**
    *   Lower Bound:  F4 * (1 - 0.10) ≤ F4_actual = -133 + 3 * MON
    *   Upper Bound:  F4 * (1 + 0.10) ≥ F4_actual = -133 + 3 * MON

**Process Constraints:**

5.  **External Isobutane-to-Olefin Ratio:** 

In [320]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample Data (replace with actual data if available)
data = {
    'ε_Alk': 2.5,  # Sample revenue per barrel of alkylate (replace with actual pricing)
    'C_o': 50,     # Sample cost per barrel of olefin feed
    'C_i': 45,     # Sample cost per barrel of isobutane
    'C_aA': 0.2,   # Sample cost per thousand pounds of acid addition
    'F_o_min': 10000,  # Sample minimum olefin feed
    'F_o_max': 20000,  # Sample maximum olefin feed
    'F_iR_min': 5000,   # Sample minimum isobutane recycle
    'F_iR_max': 15000,  # Sample maximum isobutane recycle
    'F_aA_min': 50,     # Sample minimum acid addition rate
    'F_aA_max': 150,    # Sample maximum acid addition rate
    'F_iM_min': 1000,   # Sample minimum isobutane makeup
    'F_iM_max': 10000,  # Sample maximum isobutane makeup
}

# Model Creation
model = pyo.ConcreteModel()

# Decision Variables
model.F_o = pyo.Var(bounds=(data['F_o_min'], data['F_o_max']))
model.F_iR = pyo.Var(bounds=(dat